In [5]:
#common code:
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import re
#week 3
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#week 4
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
#week 5
import requests
from bs4 import BeautifulSoup
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#week 6
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, RandomForestRegressor
from sklearn.svm import LinearSVC, SVC, LinearSVR
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.datasets import make_classification
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier


In [6]:
train = pd.read_csv('clean_train.csv')

In [7]:
train.head() #preview data

,animalid,name,datetime,outcometype,outcomesubtype,animaltype,sexuponoutcome,ageuponoutcome,breed,color
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,unknown,Dog,Neutered Male,365,Shetland Sheepdog Mix,Brown/White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,365,Domestic Shorthair Mix,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,730,Pit Bull Mix,Blue/White
3,A683430,Unnamed,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,21,Domestic Shorthair Mix,Blue Cream
4,A667013,Unnamed,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,730,Lhasa Apso/Miniature Poodle,Tan


In [53]:
train['outcometype'].value_counts(normalize=True) #baseline

Adoption           0.403530
Transfer           0.352231
Return_to_owner    0.179301
Euthanasia         0.058156
Died               0.006782
Name: outcometype, dtype: float64

In [48]:
features = ['outcomesubtype', 'animaltype', 
            'sexuponoutcome', 'ageuponoutcome', 'breed', 'color']
X = train[features]
X = pd.get_dummies(X, columns=['outcomesubtype', 'animaltype', 'sexuponoutcome', 'breed', 'color'],
                  drop_first=True)
y = train['outcometype']
y = pd.get_dummies(y)

In [49]:
X.shape

(26687, 1766)

My dummification made quite a few more columns (probably names mostly) in X. I don't need to scale because my only integer value is all on the same scale (days).

In [50]:
y.shape

(26687, 5)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=17,stratify=y)

In [52]:
for num in [X_train, X_test, y_train, y_test]: #check shapes
    print(num.shape)

(20015, 1766)
(6672, 1766)
(20015, 5)
(6672, 5)


My first model will be a Random Forest classifier.

In [54]:
rf = RandomForestClassifier()
params = {
    'n_estimators': [75,100,125],
    'max_depth': [None, 3, 5, 7],
    'max_features': ['auto', 'log2']
}
gs = GridSearchCV(rf, params, n_jobs=8, verbose=1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done 120 out of 120 | elapsed:  2.7min finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=8,
             param_grid={'max_depth': [None, 3, 5, 7],
                         'max_features': ['auto', 'log2'],
                         'n_estimators': [75, 100, 125]},
             verbose=1)

In [56]:
gs.best_params_

{'max_depth': None, 'max_features': 'auto', 'n_estimators': 100}

In [57]:
gs.score(X_train, y_train)

0.9662253310017487

In [58]:
gs.score(X_test, y_test)

0.8535671462829736

Hmmm pretty significant overfit from this grid search. Training score was ~97% but testing was only 85%. I'm going to force the grid search into some hyperparameters that should decrease variance.

In [ ]:
rf = RandomForestClassifier()
params = {
    'n_estimators': [125,150], #best n_estimator (repeated tests)
    'max_depth': [75], #remove None from options for max depth
    'min_samples_split': [None,3,4,5],
    'max_features': ['auto']
}
gs = GridSearchCV(rf, params, n_jobs=8, verbose=1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


In [60]:
gs.best_params_

{'max_depth': 100,
 'max_features': 'auto',
 'min_samples_split': 5,
 'n_estimators': 150}

In [61]:
gs.score(X_train, y_train)

0.9376467649263053

In [62]:
gs.score(X_test, y_test)

0.8610611510791367

My next model will be K-nearest neighbors model.

In [ ]:

knn = KNeighborsClassifier()
k_params = {
    
}
gs_knn = GridSearchCV(knn, k_params, n_jobs=8, verbose=1)